In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

**Create fully connected neuralnet**

In [63]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [39]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [64]:
# Hyperparameters
input_size = 784 
num_classes = 10 
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [ ]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [65]:
# Initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [50]:
model

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [66]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        
        # forward
        scores = model(data) 
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
    print(f'EPOCH {epoch}')
    check_accuracy(train_loader, model)
    check_accuracy(test_loader, model)

EPOCH 0
Checking accuracy on training data
Got 55921 / 60000 with accuracy 93.20
Checking accuracy on test data
Got 9329 / 10000 with accuracy 93.29
EPOCH 1
Checking accuracy on training data
Got 57022 / 60000 with accuracy 95.04
Checking accuracy on test data
Got 9473 / 10000 with accuracy 94.73
EPOCH 2
Checking accuracy on training data
Got 57751 / 60000 with accuracy 96.25
Checking accuracy on test data
Got 9614 / 10000 with accuracy 96.14
EPOCH 3
Checking accuracy on training data
Got 58090 / 60000 with accuracy 96.82
Checking accuracy on test data
Got 9645 / 10000 with accuracy 96.45
EPOCH 4
Checking accuracy on training data
Got 58227 / 60000 with accuracy 97.05
Checking accuracy on test data
Got 9664 / 10000 with accuracy 96.64
EPOCH 5
Checking accuracy on training data
Got 58609 / 60000 with accuracy 97.68
Checking accuracy on test data
Got 9699 / 10000 with accuracy 96.99
EPOCH 6
Checking accuracy on training data
Got 58753 / 60000 with accuracy 97.92
Checking accuracy on test

In [48]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
